# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('/Users/Zhiyang/DSI/scaling_to_production/05_src/data/fires/forestfires.csv', header = None, names = columns))


In [2]:
fires_dt.sample(n = 5)

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
128,3,5,sep,fri,93.5,149.3,728.6,8.1,17.2,43,3.1,0,0
65,2,2,aug,mon,91.1,103.2,638.8,5.8,23.1,31,3.1,0,0
464,6,5,feb,tue,75.1,4.4,16.2,1.9,4.6,82,6.3,0,5.39
7,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0,0
12,7,5,sep,sat,92.8,73.2,713,22.6,19.3,38,4,0,0


In [27]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
import pandas as pd

from sklearn.model_selection import cross_validate


# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop(columns = 'area')
Y = fires_dt['area']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [5]:
# preproc1
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']


preproc1 = ColumnTransformer(
    transformers=[('num_transform', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                                    ('scaler', StandardScaler())]), numeric_features),
                  ('cat_transform', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                             ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)])


In [6]:
preproc1

ColumnTransformer(transformers=[('num_transform',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat_transform',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [7]:
# preproc2
from sklearn.preprocessing import PowerTransformer

preproc2 = ColumnTransformer(
    transformers=[('num_transform', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                                    ('scaler', StandardScaler()),
                                                    ('power', PowerTransformer())]), numeric_features),
                  ('cat_transform', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)])

In [8]:
preproc2

ColumnTransformer(transformers=[('num_transform',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler()),
                                                 ('power',
                                                  PowerTransformer())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat_transform',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['month', 'day'])])

In [9]:
preproc1.get_params()

{'n_jobs': None,
 'remainder': 'drop',
 'sparse_threshold': 0.3,
 'transformer_weights': None,
 'transformers': [('num_transform',
   Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())]),
   ['coord_x',
    'coord_y',
    'ffmc',
    'dmc',
    'dc',
    'isi',
    'temp',
    'rh',
    'wind',
    'rain']),
  ('cat_transform',
   Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                   ('onehot', OneHotEncoder(handle_unknown='ignore'))]),
   ['month', 'day'])],
 'verbose': False,
 'verbose_feature_names_out': True,
 'num_transform': Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())]),
 'cat_transform': Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                 ('onehot', OneHotEncoder(handle_unknown='ignore'))]),
 'num_transform__memory': None,
 'num_transform__steps': [('imputer', SimpleImputer()),
  ('scaler', StandardScaler())],
 'num_transform__verbose': False,
 'num_transf

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [10]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline([('preprocessing', preproc1),('regressor', LinearRegression())])

In [11]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline([('preprocessing', preproc2),('regressor', LinearRegression())])

In [21]:
# Pipeline C = preproc1 + advanced model
# I am going to use random forest
from sklearn.ensemble import RandomForestRegressor

pipeline_C = Pipeline([('preprocessing', preproc1),('regressor', RandomForestRegressor())])

In [22]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline([('preprocessing', preproc2),('regressor', RandomForestRegressor())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [14]:
scoring = {
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'r2': 'r2',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'explained_variance': 'explained_variance'
}

In [16]:
# Pipeline A = preproc1 + baseline
# pipeline_A = Pipeline([('preprocessing', preproc1),('regressor', LinearRegression())])

param_grid_A = {'regressor__fit_intercept': [True, False]}
#Grid Search with Cross-Validation
grid_search_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, 
                             scoring=scoring, n_jobs=-1, refit='neg_mean_squared_error')
grid_search_A.fit(X_train, Y_train)


result_A = pd.DataFrame(grid_search_A.cv_results_)

result_A[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
             'param_regressor__fit_intercept', 'params',
             'mean_test_neg_mean_squared_error', 'std_test_neg_mean_squared_error',
             'mean_test_r2', 'std_test_r2',
             'mean_test_neg_mean_absolute_error', 'std_test_neg_mean_absolute_error',
             'mean_test_explained_variance', 'std_test_explained_variance',
             'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__fit_intercept,params,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,mean_test_r2,std_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,mean_test_explained_variance,std_test_explained_variance,rank_test_neg_mean_squared_error
0,0.029616,0.010257,0.037139,0.030719,True,{'regressor__fit_intercept': True},-5330.312715,7895.924904,-0.659240,0.583621,-26.696084,5.322146,-0.534382,0.473752,1
1,0.034800,0.007493,0.019925,0.009781,False,{'regressor__fit_intercept': False},-5333.159695,7894.627583,-0.662116,0.584026,-26.726741,5.300152,-0.537263,0.474983,2


In [17]:
print(grid_search_A.best_params_)
print(grid_search_A.best_score_)

{'regressor__fit_intercept': True}
-5330.312714599113


In [28]:
best_params_A = grid_search_A.best_params_

pipeline_A.set_params(**best_params_A)

res_A = cross_validate(pipeline_A, X_train, Y_train, cv = 5, scoring = scoring)
res_A = pd.DataFrame(res_A).assign(experiment = 1)
res_A

,fit_time,score_time,test_neg_mean_squared_error,test_r2,test_neg_mean_absolute_error,test_explained_variance,experiment
0,0.031752,0.040662,-1527.110655,-1.504761,-28.756917,-1.011451,1
1,0.014895,0.010020,-1068.287489,-0.469641,-23.954334,-0.368697,1
2,0.020527,0.008194,-21104.765172,-0.014015,-36.019473,0.015711,1
3,0.016701,0.006325,-1999.875116,-0.136282,-24.046700,-0.136018,1
4,0.013737,0.006604,-951.525141,-1.171501,-20.702994,-1.171454,1


In [30]:
res_A.mean()

fit_time                           0.019522
score_time                         0.014361
test_neg_mean_squared_error    -5330.312715
test_r2                           -0.659240
test_neg_mean_absolute_error     -26.696084
test_explained_variance           -0.534382
experiment                         1.000000
dtype: float64

In [18]:
# Pipeline B = preproc2 + baseline
# pipeline_B = Pipeline([('preprocessing', preproc2),('regressor', LinearRegression())])

param_grid_B = {'regressor__fit_intercept': [True, False]}
#Grid Search with Cross-Validation
grid_search_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring=scoring, n_jobs=-1, refit='neg_mean_squared_error')
grid_search_B.fit(X_train, Y_train)


result_B = pd.DataFrame(grid_search_B.cv_results_)

result_B[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
          'param_regressor__fit_intercept', 'params',
          'mean_test_neg_mean_squared_error', 'std_test_neg_mean_squared_error',
          'mean_test_r2', 'std_test_r2',
          'mean_test_neg_mean_absolute_error', 'std_test_neg_mean_absolute_error',
          'mean_test_explained_variance', 'std_test_explained_variance',
          'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__fit_intercept,params,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,mean_test_r2,std_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,mean_test_explained_variance,std_test_explained_variance,rank_test_neg_mean_squared_error
1,0.069456,0.005658,0.021969,0.003649,False,{'regressor__fit_intercept': False},-5268.044975,7867.083251,-0.579571,0.661087,-26.874138,5.967052,-0.438028,0.452191,1
0,0.085619,0.012787,0.021955,0.006825,True,{'regressor__fit_intercept': True},-5268.146330,7867.028736,-0.579748,0.661048,-26.879089,5.962375,-0.438185,0.452181,2


In [19]:
print(grid_search_B.best_params_)
print(grid_search_B.best_score_)

{'regressor__fit_intercept': False}
-5268.044974835361


In [29]:
best_params_B = grid_search_B.best_params_

pipeline_B.set_params(**best_params_B)

res_B = cross_validate(pipeline_B, X_train, Y_train, cv = 5, scoring = scoring)
res_B = pd.DataFrame(res_B).assign(experiment = 1)
res_B

,fit_time,score_time,test_neg_mean_squared_error,test_r2,test_neg_mean_absolute_error,test_explained_variance,experiment
0,0.108462,0.020176,-1731.547033,-1.840077,-31.011861,-1.262003,1
1,0.037709,0.007742,-1089.236688,-0.498461,-24.653072,-0.408656,1
2,0.039836,0.007858,-20976.544899,-0.007855,-36.139312,0.021789,1
3,0.044712,0.009527,-1897.225804,-0.077960,-23.234443,-0.076842,1
4,0.040893,0.008686,-645.670451,-0.473502,-19.332001,-0.464425,1


In [31]:
res_B.mean()

fit_time                           0.054322
score_time                         0.010798
test_neg_mean_squared_error    -5268.044975
test_r2                           -0.579571
test_neg_mean_absolute_error     -26.874138
test_explained_variance           -0.438028
experiment                         1.000000
dtype: float64

In [32]:
# Pipeline C = preproc1 + advanced model
# I am going to use random forest
# pipeline_C = Pipeline([('preprocessing', preproc1),('regressor', RandomForestRegressor())])

#the parameter grid for GridSearchCV
param_grid_C = {
    'regressor__n_estimators': [50, 100, 200], #I tried 300 it's too much for my laptop
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['auto', 'sqrt', 'log2']
}

#Grid Search with Cross-Validation
grid_search_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring=scoring, refit='neg_mean_squared_error')
grid_search_C.fit(X_train, Y_train)

result_C = pd.DataFrame(grid_search_C.cv_results_)

result_C[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
          'param_regressor__n_estimators', 'param_regressor__max_depth', 
          'param_regressor__min_samples_split', 'param_regressor__min_samples_leaf', 'params',
          'mean_test_neg_mean_squared_error', 'std_test_neg_mean_squared_error', 'mean_test_r2', 'std_test_r2',
          'mean_test_neg_mean_absolute_error', 'std_test_neg_mean_absolute_error',
          'mean_test_explained_variance', 'std_test_explained_variance',
          'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/pipeline.py", line

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_estimators,param_regressor__max_depth,param_regressor__min_samples_split,param_regressor__min_samples_leaf,params,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,mean_test_r2,std_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,mean_test_explained_variance,std_test_explained_variance,rank_test_neg_mean_squared_error
138,0.086728,0.005951,0.012163,0.001195,50,10,5,1,"{'regressor__max_depth': 10, 'regressor__max_f...",-5038.596766,7806.589403,-0.277055,0.500273,-22.013826,7.704260,-0.159987,0.414907,1
34,0.166010,0.032607,0.012974,0.002016,100,None,10,1,"{'regressor__max_depth': None, 'regressor__max...",-5067.836032,7761.568509,-0.356570,0.649790,-22.622746,7.827271,-0.218823,0.515744,2
63,0.191684,0.069373,0.018338,0.007275,50,None,2,2,"{'regressor__max_depth': None, 'regressor__max...",-5071.443280,7837.470579,-0.326292,0.631225,-21.932483,8.184790,-0.198403,0.523388,3
211,0.143111,0.006679,0.013236,0.001070,100,20,5,4,"{'regressor__max_depth': 20, 'regressor__max_f...",-5075.398138,8045.221962,-0.194479,0.359692,-21.965796,7.460174,-0.081081,0.280782,4
79,0.171193,0.046475,0.014197,0.001838,100,None,10,4,"{'regressor__max_depth': None, 'regressor__max...",-5076.689577,7998.845694,-0.230535,0.358604,-22.253285,7.432930,-0.110078,0.256179,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,0.012554,0.000808,0.000000,0.000000,50,20,5,2,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
175,0.010547,0.001663,0.000000,0.000000,100,20,5,2,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
176,0.010712,0.000949,0.000000,0.000000,200,20,5,2,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
169,0.019217,0.007866,0.000000,0.000000,100,20,10,1,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217


In [33]:
print(grid_search_C.best_params_)
print(grid_search_C.best_score_)

{'regressor__max_depth': 10, 'regressor__max_features': 'log2', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 50}
-5038.596766333328


In [34]:
best_params_C = grid_search_C.best_params_

pipeline_C.set_params(**best_params_C)

res_C = cross_validate(pipeline_C, X_train, Y_train, cv = 5, scoring = scoring)
res_C = pd.DataFrame(res_C).assign(experiment = 1)
res_C

,fit_time,score_time,test_neg_mean_squared_error,test_r2,test_neg_mean_absolute_error,test_explained_variance,experiment
0,0.112860,0.015340,-1601.219118,-1.626314,-22.840182,-1.254661,1
1,0.082874,0.012231,-1334.045635,-0.835244,-23.663140,-0.571088,1
2,0.081849,0.010876,-20857.341586,-0.002127,-36.856669,0.025007,1
3,0.082942,0.011707,-1933.428374,-0.098529,-19.603009,-0.096392,1
4,0.114006,0.014871,-327.497738,0.252609,-12.928657,0.372625,1


In [35]:
res_C.mean()
std_cv_A = np.std(cv_scores_A)

fit_time                           0.094906
score_time                         0.013005
test_neg_mean_squared_error    -5210.706490
test_r2                           -0.461921
test_neg_mean_absolute_error     -23.178331
test_explained_variance           -0.304902
experiment                         1.000000
dtype: float64

In [36]:
# Pipeline D = preproc2 + advanced model
# also random forest
# pipeline_D = Pipeline([('preprocessing', preproc2),('regressor', RandomForestRegressor())])

#the parameter grid for GridSearchCV
param_grid_D = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['auto', 'sqrt', 'log2']
}

#Grid Search with Cross-Validation
grid_search_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring=scoring, n_jobs=-1, refit='neg_mean_squared_error')
grid_search_D.fit(X_train, Y_train)

result_D = pd.DataFrame(grid_search_D.cv_results_)

result_D[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
          'param_regressor__n_estimators', 'param_regressor__max_depth', 
          'param_regressor__min_samples_split', 'param_regressor__min_samples_leaf', 'params',
          'mean_test_neg_mean_squared_error', 'std_test_neg_mean_squared_error', 'mean_test_r2', 'std_test_r2',
          'mean_test_neg_mean_absolute_error', 'std_test_neg_mean_absolute_error',
          'mean_test_explained_variance', 'std_test_explained_variance',
          'rank_test_neg_mean_squared_error']].sort_values('rank_test_neg_mean_squared_error')

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/pipeline.py", line

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_estimators,param_regressor__max_depth,param_regressor__min_samples_split,param_regressor__min_samples_leaf,params,mean_test_neg_mean_squared_error,std_test_neg_mean_squared_error,mean_test_r2,std_test_r2,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,mean_test_explained_variance,std_test_explained_variance,rank_test_neg_mean_squared_error
233,0.957658,0.163847,0.051725,0.021637,200,20,10,2,"{'regressor__max_depth': 20, 'regressor__max_f...",-5041.279667,7907.960960,-0.244959,0.535342,-21.936074,7.622683,-0.120537,0.425510,1
118,0.445464,0.049118,0.036186,0.011230,100,10,2,2,"{'regressor__max_depth': 10, 'regressor__max_f...",-5055.763845,7812.991447,-0.328329,0.753687,-22.136491,8.093642,-0.189133,0.595821,2
235,0.552684,0.026679,0.043676,0.010627,100,20,2,4,"{'regressor__max_depth': 20, 'regressor__max_f...",-5073.616864,7993.874967,-0.235133,0.341079,-22.105488,7.436206,-0.122998,0.240835,3
142,0.400018,0.054486,0.042075,0.010786,100,10,10,1,"{'regressor__max_depth': 10, 'regressor__max_f...",-5074.067674,7840.282436,-0.313815,0.610526,-22.312546,7.708572,-0.176210,0.466746,4
159,0.222612,0.039555,0.020569,0.002237,50,10,10,4,"{'regressor__max_depth': 10, 'regressor__max_f...",-5075.411295,7985.435769,-0.248727,0.417353,-21.770025,7.621430,-0.139957,0.310666,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,0.086275,0.020525,0.000000,0.000000,50,20,5,4,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
184,0.086895,0.019823,0.000000,0.000000,100,20,5,4,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
185,0.092087,0.014853,0.000000,0.000000,200,20,5,4,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
179,0.106600,0.035185,0.000000,0.000000,200,20,10,2,"{'regressor__max_depth': 20, 'regressor__max_f...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217


In [37]:
print(grid_search_D.best_params_)
print(grid_search_D.best_score_)

{'regressor__max_depth': 20, 'regressor__max_features': 'log2', 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 200}
-5041.279667336641


In [38]:
best_params_D = grid_search_D.best_params_

pipeline_D.set_params(**best_params_D)

res_D = cross_validate(pipeline_D, X_train, Y_train, cv = 5, scoring = scoring)
res_D = pd.DataFrame(res_D).assign(experiment = 1)
res_D

,fit_time,score_time,test_neg_mean_squared_error,test_r2,test_neg_mean_absolute_error,test_explained_variance,experiment
0,0.371746,0.017868,-1496.554640,-1.454643,-23.363055,-1.046404,1
1,0.755050,0.031680,-903.547624,-0.243009,-20.364703,-0.105819,1
2,0.594356,0.035017,-20952.061666,-0.006678,-36.449352,0.019978,1
3,0.387441,0.025412,-1846.746225,-0.049278,-18.813182,-0.046112,1
4,0.347190,0.022590,-362.630388,0.172431,-13.775601,0.290785,1


In [39]:
res_D.mean()

fit_time                           0.491157
score_time                         0.026513
test_neg_mean_squared_error    -5112.308108
test_r2                           -0.316235
test_neg_mean_absolute_error     -22.553179
test_explained_variance           -0.177515
experiment                         1.000000
dtype: float64

# Evaluate

+ Which model has the best performance?

In [47]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score

best_model_A = grid_search_A.best_estimator_
best_model_A.fit(X_train, Y_train)
Y_pred = best_model_A.predict(X_test)

test_mse = mean_squared_error(Y_test, Y_pred)
test_r2 = r2_score(Y_test, Y_pred)
test_mae = mean_absolute_error(Y_test, Y_pred)
test_explained_variance = explained_variance_score(Y_test, Y_pred)

test_performance = {
    'Mean Squared Error': test_mse,
    'R^2 Score': test_r2,
    'Mean Absolute Error': test_mae,
    'Explained Variance': test_explained_variance
}

print(test_performance)

# Why I got the error????? which column is non-numeric????


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'X'

# Export

+ Save the best performing model to a pickle file.

In [ ]:
import joblib

model_filename = 'best_model.pkl'
#best model = ????????????
joblib.dump(best_model, model_filename)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [49]:
import shap
import matplotlib.pyplot as plt

best_model = joblib.load('best_model.pkl')

explainer = shap.TreeExplainer(best_model.named_steps['regressor'])
shap_values = explainer.shap_values(best_model.named_steps['preprocessing'].transform(X_test))

observation_index = 0
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[observation_index], X_test.iloc[observation_index], feature_names=X_test.columns)


shap.summary_plot(shap_values, X_test, feature_names=X_test.columns)


In [ ]:
shap.plots.waterfall(shap_values[1])

In [ ]:
shap.plots.beeswarm(shap_values)

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.